# Transformer

Transformer의 핵심 구조인 Multi-head Attention을 구현하는 실습입니다.
1. Multi-head attention 및 self-attention 구현.
2. 각 과정에서 일어나는 연산과 input/output 형태 이해.

### 필요 패키지 import

In [1]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

## Req. 2-1 Multi-head self-attention 구조 익히기

### **데이터 전처리**
vocab_size 100인 가상의 시퀀스 데이터를 생성합니다. 

각 데이터에 할당된 숫자는 tokenizing과 정수화가 이뤄진 형태입니다.

In [2]:
pad_id = 0
vocab_size = 100

data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

In [3]:
# 길이 맞춰주기 위해 패딩합니다.
def padding(data):
  max_len = len(max(data, key=len))
  print(f"Maximum sequence length: {max_len}")

  for i, seq in enumerate(tqdm(data)):
    if len(seq) < max_len:
      data[i] = seq + [pad_id] * (max_len - len(seq))

  return data, max_len

In [4]:
data, max_len = padding(data)

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 79437.58it/s]


In [5]:
data

[[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0],
 [60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [77,
  65,
  51,
  77,
  19,
  15,
  35,
  19,
  23,
  97,
  50,
  46,
  53,
  42,
  45,
  91,
  66,
  3,
  43,
  10],
 [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0],
 [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]]

### Hyperparameter 세팅 및 embedding

In [6]:
d_model = 512  # model의 hidden size
num_heads = 8  # head의 개수

# d_model이 입력을 projection 시킬 임베딩 space의 차원이므로, num_heads로 나누어 떨어져야 한다.

In [7]:
embedding = nn.Embedding(vocab_size, d_model)

# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_model)

In [8]:
print(batch_emb)
print(batch_emb.shape)

tensor([[[-0.7367,  0.7097,  1.7420,  ..., -0.6935,  0.6663,  0.9574],
         [-0.0627, -1.5894, -1.7091,  ...,  0.4887, -0.4114,  0.6533],
         [ 1.7523, -1.3457,  0.6836,  ...,  0.0589,  0.1447,  0.8339],
         ...,
         [-0.1690,  1.4675,  0.4294,  ...,  0.4046,  0.8386, -0.8755],
         [-0.1690,  1.4675,  0.4294,  ...,  0.4046,  0.8386, -0.8755],
         [-0.1690,  1.4675,  0.4294,  ...,  0.4046,  0.8386, -0.8755]],

        [[-0.7866,  0.6204, -0.2504,  ..., -0.0363,  0.7677,  0.3478],
         [-0.3669,  0.0494,  1.9281,  ..., -0.4636,  1.6363, -0.5044],
         [-1.6859,  2.1032, -1.2072,  ...,  0.7961,  1.6086,  1.5415],
         ...,
         [-0.1690,  1.4675,  0.4294,  ...,  0.4046,  0.8386, -0.8755],
         [-0.1690,  1.4675,  0.4294,  ...,  0.4046,  0.8386, -0.8755],
         [-0.1690,  1.4675,  0.4294,  ...,  0.4046,  0.8386, -0.8755]],

        [[-1.3682,  0.5439, -0.5723,  ...,  1.3878, -1.2619, -0.6299],
         [ 0.3999, -0.6068,  0.7600,  ...,  0

### Linear projection & 여러 head로 나누기

Multi-head attention 내에서 쓰이는 linear projection matrix들을 정의합니다.

In [9]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

In [10]:
w_0 = nn.Linear(d_model, d_model)

In [11]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 512])
torch.Size([10, 20, 512])
torch.Size([10, 20, 512])


Q, k, v를 `num_head`개의 차원 분할된 여러 vector로 만듭니다.

- 이론적으로는 multi-head attention을 수행하면 input을 각각 다른 head 개수만큼의 Wq, Wk, Wv로 linear transformation 해서 각각 여러번의 attention 수행한 후 concat 한 후 linear transformation 수행해준다
- 구현에서는 Wq, Wk, Wv 한 개씩
- 실제 `attention is all you need` 논문의 구현 예시는 Query vector 한개를 dim으로 쪼개서 진행한다

In [12]:
batch_size = q.shape[0]
d_k = d_model // num_heads

# num_heads * d_k로 쪼갠다
q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])


In [13]:
# num_heads를 밖으로 뺌으로써
# 각 head가 (L, d_k) 만큼의 matrix를 가지고 self-attention 수행

q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])


### Scaled dot-product self-attention 구현

각 head에서 실행되는 self-attetion 과정입니다.

In [14]:
# shape - (L, L)
# 같은 sequence 내에 서로 다른 token들에게 얼마나 가중치를 두고 attention을 해야하는가
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
# softmax - row-wise이기 때문에 dim은 -1
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

print(attn_dists)
print(attn_dists.shape)

tensor([[[[0.0702, 0.0502, 0.0387,  ..., 0.0497, 0.0497, 0.0497],
          [0.0532, 0.0500, 0.0477,  ..., 0.0604, 0.0604, 0.0604],
          [0.0559, 0.0422, 0.0559,  ..., 0.0459, 0.0459, 0.0459],
          ...,
          [0.0675, 0.0890, 0.0800,  ..., 0.0404, 0.0404, 0.0404],
          [0.0675, 0.0890, 0.0800,  ..., 0.0404, 0.0404, 0.0404],
          [0.0675, 0.0890, 0.0800,  ..., 0.0404, 0.0404, 0.0404]],

         [[0.0484, 0.0491, 0.0437,  ..., 0.0626, 0.0626, 0.0626],
          [0.0702, 0.0328, 0.0384,  ..., 0.0580, 0.0580, 0.0580],
          [0.0303, 0.0229, 0.0431,  ..., 0.0600, 0.0600, 0.0600],
          ...,
          [0.0353, 0.0449, 0.0512,  ..., 0.0530, 0.0530, 0.0530],
          [0.0353, 0.0449, 0.0512,  ..., 0.0530, 0.0530, 0.0530],
          [0.0353, 0.0449, 0.0512,  ..., 0.0530, 0.0530, 0.0530]],

         [[0.0704, 0.0454, 0.0454,  ..., 0.0426, 0.0426, 0.0426],
          [0.0232, 0.0596, 0.0782,  ..., 0.0435, 0.0435, 0.0435],
          [0.0473, 0.0489, 0.0798,  ..., 0

In [15]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

torch.Size([10, 8, 20, 64])


### 각 head의 결과물 병합

각 head의 결과물을 concat하고 동일 차원으로 linear projection합니다.

In [16]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

torch.Size([10, 20, 512])


In [17]:
# w_0 : (d_model, d_model)
# 서로 다른 의미로 foucsing 된 각 head의 self-attention 정보들을 합쳐주는 역할 수행
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

tensor([[[-0.0043, -0.1327,  0.0959,  ...,  0.1094, -0.1679, -0.0445],
         [ 0.0996, -0.0957,  0.1594,  ...,  0.1120, -0.1612,  0.0158],
         [ 0.0684, -0.0878,  0.1038,  ...,  0.0545, -0.1531,  0.0510],
         ...,
         [ 0.0721, -0.1334,  0.1440,  ...,  0.1428, -0.1771, -0.0444],
         [ 0.0721, -0.1334,  0.1440,  ...,  0.1428, -0.1771, -0.0444],
         [ 0.0721, -0.1334,  0.1440,  ...,  0.1428, -0.1771, -0.0444]],

        [[-0.0793, -0.3827,  0.0999,  ...,  0.3194, -0.4645,  0.0162],
         [-0.0429, -0.3748,  0.0616,  ...,  0.3099, -0.4279,  0.0227],
         [-0.0572, -0.3540,  0.0572,  ...,  0.3466, -0.4188, -0.0042],
         ...,
         [-0.0685, -0.4596,  0.0783,  ...,  0.2936, -0.3906, -0.0609],
         [-0.0685, -0.4596,  0.0783,  ...,  0.2936, -0.3906, -0.0609],
         [-0.0685, -0.4596,  0.0783,  ...,  0.2936, -0.3906, -0.0609]],

        [[ 0.0295, -0.3399,  0.0336,  ...,  0.2284, -0.2756,  0.0141],
         [ 0.0222, -0.2637,  0.0246,  ...,  0

## Req. 2-2 Multi-head self-attention 모듈 클래스 구현

위의 과정을 모두 합쳐 하나의 Multi-head attention 모듈 class를 구현하겠습니다.

아래 코드의 TODO 부분을 채워주세요.

In [22]:
class MultiheadAttention(nn.Module):
  def __init__(self):
    super(MultiheadAttention, self).__init__()

    # Q, K, V learnable matrices
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    # Linear projection for concatenated outputs
    self.w_0 = nn.Linear(d_model, d_model)

  # scaled-dot product attention
  def self_attention(self, q, k, v):
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
    attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

    attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

    return attn_values

  def forward(self, q, k, v):
    batch_size = q.shape[0]

    # linear projection
    ################################################################################
    # TODO 1: Implement the forward pass for linear projection.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    q = self.w_q(q)
    k = self.w_k(k)
    v = self.w_v(v)

    # head만큼 쪼개준다
    ################################################################################
    # TODO 2: Implement the forward pass for split head.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    q = q.view(batch_size, -1, num_heads, d_k)
    k = k.view(batch_size, -1, num_heads, d_k)
    v = v.view(batch_size, -1, num_heads, d_k)

    # 각 head가 (L, d_k)의 matrix를 담당하도록 만든다
    q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
    k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
    v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

    attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)
    attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, d_model)  # (B, L, num_heads, d_k) => (B, L, d_model)

    return self.w_0(attn_values)

In [23]:
multihead_attn = MultiheadAttention()

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [24]:
print(outputs)
print(outputs.shape)  # (batch_size, length, d_model)

tensor([[[-0.1881,  0.1743,  0.0278,  ...,  0.1217, -0.0694, -0.0161],
         [-0.2214,  0.1663, -0.0205,  ...,  0.0765,  0.0010, -0.0358],
         [-0.1969,  0.2258,  0.0189,  ...,  0.1599,  0.0009, -0.0354],
         ...,
         [-0.1963,  0.1910, -0.0243,  ...,  0.1073, -0.0483,  0.0027],
         [-0.1963,  0.1910, -0.0243,  ...,  0.1073, -0.0483,  0.0027],
         [-0.1963,  0.1910, -0.0243,  ...,  0.1073, -0.0483,  0.0027]],

        [[-0.1842, -0.1065,  0.1868,  ...,  0.0323, -0.1754,  0.1611],
         [-0.1118, -0.0972,  0.2141,  ...,  0.0993, -0.1083,  0.1668],
         [-0.2151, -0.1280,  0.2129,  ..., -0.0043, -0.1942,  0.1965],
         ...,
         [-0.1780, -0.1197,  0.1646,  ...,  0.0247, -0.1798,  0.1981],
         [-0.1780, -0.1197,  0.1646,  ...,  0.0247, -0.1798,  0.1981],
         [-0.1780, -0.1197,  0.1646,  ...,  0.0247, -0.1798,  0.1981]],

        [[-0.0263, -0.0193,  0.1181,  ...,  0.0981, -0.2188,  0.1416],
         [-0.1618, -0.0126,  0.1372,  ...,  0